# Textklassifizierungsaufgabe

In diesem Modul beginnen wir mit einer einfachen Textklassifizierungsaufgabe basierend auf dem **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**-Datensatz: Wir werden Nachrichtenüberschriften in eine von vier Kategorien einordnen: Welt, Sport, Wirtschaft und Wissenschaft/Technik.

## Der Datensatz

Um den Datensatz zu laden, verwenden wir die **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**-API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Wir können nun auf die Trainings- und Testteile des Datensatzes zugreifen, indem wir `dataset['train']` und `dataset['test']` verwenden:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Lassen Sie uns die ersten 10 neuen Schlagzeilen aus unserem Datensatz ausdrucken:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Textvektorisierung

Nun müssen wir Text in **Zahlen** umwandeln, die als Tensoren dargestellt werden können. Wenn wir eine Wortebene-Darstellung möchten, müssen wir zwei Dinge tun:

* Einen **Tokenizer** verwenden, um den Text in **Token** zu zerlegen.
* Ein **Vokabular** dieser Token erstellen.

### Begrenzung der Vokabulargröße

Im Beispiel des AG News-Datensatzes ist die Vokabulargröße ziemlich groß, mehr als 100.000 Wörter. Allgemein gesprochen benötigen wir keine Wörter, die selten im Text vorkommen — nur wenige Sätze enthalten sie, und das Modell wird nicht von ihnen lernen. Daher ist es sinnvoll, die Vokabulargröße auf eine kleinere Anzahl zu begrenzen, indem ein Argument an den Vektorisierer-Konstruktor übergeben wird:

Beide dieser Schritte können mit der **TextVectorization**-Schicht durchgeführt werden. Lassen Sie uns das Vektorisierungsobjekt instanziieren und anschließend die `adapt`-Methode aufrufen, um den gesamten Text zu durchlaufen und ein Vokabular zu erstellen:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Hinweis**: Wir verwenden nur einen Teil des gesamten Datensatzes, um ein Vokabular zu erstellen. Dies tun wir, um die Ausführungszeit zu verkürzen und Sie nicht warten zu lassen. Allerdings gehen wir das Risiko ein, dass einige Wörter aus dem gesamten Datensatz nicht in das Vokabular aufgenommen werden und während des Trainings ignoriert werden. Die Verwendung der gesamten Vokabulargröße und das Durchlaufen des gesamten Datensatzes während `adapt` sollte die endgültige Genauigkeit erhöhen, jedoch nicht signifikant.

Nun können wir auf das tatsächliche Vokabular zugreifen:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Mit dem Vektorisierer können wir problemlos jeden Text in eine Zahlenmenge kodieren:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words Textdarstellung

Da Wörter Bedeutung vermitteln, können wir manchmal die Bedeutung eines Textes allein durch die Betrachtung der einzelnen Wörter erkennen, unabhängig von ihrer Reihenfolge im Satz. Zum Beispiel deuten beim Klassifizieren von Nachrichten Wörter wie *Wetter* und *Schnee* wahrscheinlich auf eine *Wettervorhersage* hin, während Wörter wie *Aktien* und *Dollar* eher auf *Finanznachrichten* hindeuten.

Die **Bag-of-words** (BoW)-Vektordarstellung ist die einfachste und am leichtesten verständliche traditionelle Vektordarstellung. Jedes Wort wird einem Vektorindex zugeordnet, und ein Vektorelement enthält die Anzahl der Vorkommen jedes Wortes in einem bestimmten Dokument.

![Bild, das zeigt, wie eine Bag-of-words-Vektordarstellung im Speicher repräsentiert wird.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Note**: Sie können sich BoW auch als die Summe aller One-Hot-encodierten Vektoren für die einzelnen Wörter im Text vorstellen.

Unten sehen Sie ein Beispiel, wie man mit der Scikit Learn Python-Bibliothek eine Bag-of-words-Darstellung erzeugen kann:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Wir können auch den Keras-Vektorisierer verwenden, den wir oben definiert haben, indem wir jede Wortnummer in eine One-Hot-Codierung umwandeln und alle diese Vektoren addieren:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Hinweis**: Es könnte Sie überraschen, dass das Ergebnis sich vom vorherigen Beispiel unterscheidet. Der Grund dafür ist, dass im Keras-Beispiel die Länge des Vektors der Größe des Vokabulars entspricht, das aus dem gesamten AG News-Datensatz erstellt wurde, während wir im Scikit-Learn-Beispiel das Vokabular spontan aus dem Beispieltext erstellt haben.


## Training des BoW-Klassifikators

Jetzt, da wir gelernt haben, wie man die Bag-of-Words-Darstellung unseres Textes erstellt, können wir einen Klassifikator trainieren, der diese verwendet. Zuerst müssen wir unser Dataset in eine Bag-of-Words-Darstellung umwandeln. Dies kann mit der `map`-Funktion auf folgende Weise erreicht werden:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Lassen Sie uns nun ein einfaches Klassifikator-Neuronales Netzwerk definieren, das eine lineare Schicht enthält. Die Eingabegröße ist `vocab_size`, und die Ausgabengröße entspricht der Anzahl der Klassen (4). Da wir eine Klassifikationsaufgabe lösen, ist die endgültige Aktivierungsfunktion **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Da wir 4 Klassen haben, ist eine Genauigkeit von über 80 % ein gutes Ergebnis.

## Einen Klassifikator als ein Netzwerk trainieren

Da der Vektorisierer ebenfalls eine Keras-Schicht ist, können wir ein Netzwerk definieren, das ihn einschließt, und es vollständig trainieren. Auf diese Weise müssen wir den Datensatz nicht mit `map` vektorisieren, sondern können den ursprünglichen Datensatz direkt an den Eingang des Netzwerks übergeben.

> **Hinweis**: Wir müssten dennoch `map` auf unseren Datensatz anwenden, um Felder aus Wörterbüchern (wie `title`, `description` und `label`) in Tupel umzuwandeln. Wenn wir die Daten jedoch von der Festplatte laden, können wir von Anfang an einen Datensatz mit der erforderlichen Struktur erstellen.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramme, Trigramme und N-Gramme

Eine Einschränkung des Bag-of-Words-Ansatzes ist, dass einige Wörter Teil von mehrwortigen Ausdrücken sind. Zum Beispiel hat das Wort 'Hot Dog' eine völlig andere Bedeutung als die Wörter 'hot' und 'dog' in anderen Kontexten. Wenn wir die Wörter 'hot' und 'dog' immer mit denselben Vektoren darstellen, kann dies unser Modell verwirren.

Um dies zu lösen, werden häufig **N-Gramm-Darstellungen** in Methoden der Dokumentklassifikation verwendet, bei denen die Häufigkeit jedes Wortes, Zwei-Wort- oder Drei-Wort-Ausdrucks eine nützliche Eigenschaft für das Training von Klassifikatoren darstellt. In Bigramm-Darstellungen fügen wir beispielsweise alle Wortpaare zusätzlich zu den ursprünglichen Wörtern dem Vokabular hinzu.

Unten sehen Sie ein Beispiel, wie man eine Bigramm-Bag-of-Words-Darstellung mit Scikit Learn erzeugt:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Der Hauptnachteil des n-Gramm-Ansatzes ist, dass die Größe des Vokabulars extrem schnell wächst. In der Praxis müssen wir die n-Gramm-Repräsentation mit einer Technik zur Dimensionsreduktion kombinieren, wie zum Beispiel *Embeddings*, die wir in der nächsten Einheit besprechen werden.

Um eine n-Gramm-Repräsentation in unserem **AG News**-Datensatz zu verwenden, müssen wir den Parameter `ngrams` an den `TextVectorization`-Konstruktor übergeben. Die Länge eines Bigramm-Vokabulars ist **deutlich größer**, in unserem Fall sind es mehr als 1,3 Millionen Tokens! Daher ist es sinnvoll, auch die Bigramm-Tokens auf eine vernünftige Anzahl zu begrenzen.

Wir könnten denselben Code wie oben verwenden, um den Klassifikator zu trainieren, allerdings wäre das sehr speicherineffizient. In der nächsten Einheit werden wir den Bigramm-Klassifikator mithilfe von Embeddings trainieren. In der Zwischenzeit kannst du mit dem Training eines Bigramm-Klassifikators in diesem Notebook experimentieren und sehen, ob du eine höhere Genauigkeit erzielen kannst.


## Automatisches Berechnen von BoW-Vektoren

Im obigen Beispiel haben wir BoW-Vektoren manuell berechnet, indem wir die One-Hot-Codierungen einzelner Wörter summiert haben. Die neueste Version von TensorFlow ermöglicht es uns jedoch, BoW-Vektoren automatisch zu berechnen, indem wir den Parameter `output_mode='count'` an den Konstruktor des Vektorisierers übergeben. Dies vereinfacht das Definieren und Trainieren unseres Modells erheblich:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termfrequenz - inverse Dokumentfrequenz (TF-IDF)

In der BoW-Darstellung werden Wortvorkommen unabhängig vom Wort selbst mit derselben Technik gewichtet. Es ist jedoch offensichtlich, dass häufige Wörter wie *a* und *in* für die Klassifikation viel weniger wichtig sind als spezialisierte Begriffe. Bei den meisten NLP-Aufgaben sind einige Wörter relevanter als andere.

**TF-IDF** steht für **Termfrequenz - inverse Dokumentfrequenz**. Es handelt sich um eine Variation des Bag-of-Words-Modells, bei der anstelle eines binären 0/1-Wertes, der das Auftreten eines Wortes in einem Dokument angibt, ein Gleitkommawert verwendet wird, der mit der Häufigkeit des Wortvorkommens im Korpus zusammenhängt.

Formal wird das Gewicht $w_{ij}$ eines Wortes $i$ im Dokument $j$ wie folgt definiert:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
wobei
* $tf_{ij}$ die Anzahl der Vorkommen von $i$ in $j$ ist, also der BoW-Wert, den wir zuvor gesehen haben
* $N$ die Anzahl der Dokumente in der Sammlung ist
* $df_i$ die Anzahl der Dokumente ist, die das Wort $i$ in der gesamten Sammlung enthalten

Der TF-IDF-Wert $w_{ij}$ steigt proportional zur Häufigkeit, mit der ein Wort in einem Dokument erscheint, und wird durch die Anzahl der Dokumente im Korpus, die das Wort enthalten, ausgeglichen. Dies hilft, den Umstand zu berücksichtigen, dass einige Wörter häufiger vorkommen als andere. Wenn beispielsweise ein Wort in *jedem* Dokument der Sammlung vorkommt, gilt $df_i=N$, und $w_{ij}=0$, und diese Begriffe würden vollständig ignoriert.

Mit Scikit Learn können Sie ganz einfach eine TF-IDF-Vektorisierung von Text erstellen:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In Keras kann die `TextVectorization`-Schicht automatisch TF-IDF-Frequenzen berechnen, indem der Parameter `output_mode='tf-idf'` übergeben wird. Lassen Sie uns den oben verwendeten Code wiederholen, um zu sehen, ob die Verwendung von TF-IDF die Genauigkeit erhöht:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Fazit

Auch wenn TF-IDF-Darstellungen Häufigkeitsgewichte für verschiedene Wörter bereitstellen, sind sie nicht in der Lage, Bedeutung oder Reihenfolge abzubilden. Wie der berühmte Linguist J. R. Firth 1935 sagte: "Die vollständige Bedeutung eines Wortes ist immer kontextabhängig, und keine Untersuchung der Bedeutung ohne Kontext kann ernst genommen werden." Später im Kurs werden wir lernen, wie man kontextuelle Informationen aus Texten mithilfe von Sprachmodellen erfasst.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
